# Error-Mitigation With Qermit

Installation:
<p><center> <code> pip install qermit </code> </center></p>
<p>Documentation and examples are available at -> <a href="https://cqcl.github.io/qermit/">https://cqcl.github.io/qermit/</a>.</p>
<p>Manual is available at -> <a href="https://cqcl.github.io/qermit/manual/">https://cqcl.github.io/qermit/manual/</a>.</p>
<p>Repository itself is available at -> <a href="https://github.com/CQCL/qermit">https://github.com/CQCL/qermit</a>.</p>

## Constructing a SPAM + Depolarising Noise Model

In [ ]:
import qiskit.providers.aer.noise as noise
from pytket.extensions.qiskit import AerBackend

def depolarizing_noise_model(n_qubits, prob_1, prob_2, prob_ro):
    # All to all atchetecture, with depolarising noise in all gates
    # and readout error.

    noise_model = noise.NoiseModel()

    # Depolarizing 2 qubit errors
    error_2 = noise.depolarizing_error(prob_2, 2)
    for edge in [[i,j] for i in range(n_qubits) for j in range(i)]:
        noise_model.add_quantum_error(error_2, ['cx'], [edge[0], edge[1]])
        noise_model.add_quantum_error(error_2, ['cx'], [edge[1], edge[0]])

    # Depolarizing 1 qubit errors
    error_1 = noise.depolarizing_error(prob_1, 1)
    for node in range(n_qubits):
        noise_model.add_quantum_error(error_1, ['h', 'rx', 'rz', 'u'], [node])
        
    # Readout errors
    probabilities = [[1-prob_ro, prob_ro],[prob_ro, 1-prob_ro]]
    error_ro = noise.ReadoutError(probabilities)
    for i in range(n_qubits):
        noise_model.add_readout_error(error_ro, [i])
        
    return noise_model

n_qubits = 5
prob_2 = 0.01

noise_model = depolarizing_noise_model(n_qubits, prob_2*0.1, prob_2, 3*prob_2)
noisy_backend = AerBackend(noise_model)

## Generate Random Circuits

In [ ]:
from pytket import Circuit
import numpy as np
from scipy.stats import unitary_group
from pytket.circuit import Unitary2qBox
from pytket.circuit.display import render_circuit_jupyter

def random_circ(n_qubits: int, depth: int, seed:int = None) -> Circuit:
    
    np.random.seed(seed)

    c = Circuit(n_qubits)

    for _ in range(depth):

        qubits = np.random.permutation([i for i in range(n_qubits)])
        qubit_pairs = [[qubits[i], qubits[i + 1]] for i in range(0, n_qubits - 1, 2)]

        for pair in qubit_pairs:

            # Generate random 4x4 unitary matrix.
            SU4 = unitary_group.rvs(4)  # random unitary in SU4
            SU4 = SU4 / (np.linalg.det(SU4) ** 0.25)
            SU4 = np.matrix(SU4)

            # Add gate corresponding to unitary.
            c.add_unitary2qbox(Unitary2qBox(SU4), *pair)

    return c

rand_circ = random_circ(4, 3)
render_circuit_jupyter(rand_circ)

## Ideal Expectation Value

In [ ]:
from pytket.pauli import Pauli, QubitPauliString
from pytket import Qubit 
from pytket.extensions.qiskit import AerStateBackend

n_qubits = 4
rand_circ = random_circ(n_qubits,n_qubits,seed=23126)

ideal_backend = AerStateBackend()

ideal_circuit = rand_circ.copy()
ideal_circuit = ideal_backend.get_compiled_circuit(ideal_circuit)

qps = QubitPauliString([Qubit(i) for i in range(n_qubits)], [Pauli.Z for i in range(n_qubits)])

print(f"Ideal expectation: {ideal_backend.get_pauli_expectation_value(ideal_circuit, qps)}")

## Noisy Expectation Value

In [ ]:
from pytket.utils import expectation_from_counts

n_shots = 1000000

compiled_circ = rand_circ.copy()
compiled_circ.measure_all()
compiled_circ = noisy_backend.get_compiled_circuit(compiled_circ, optimisation_level=0)

result = noisy_backend.run_circuit(compiled_circ, n_shots=n_shots)
print(f"Noisy expectation: {expectation_from_counts(result.get_counts())}")

## Error Mitigation With ZNE

In [ ]:
from qermit.zero_noise_extrapolation import gen_ZNE_MitEx, Fit

zne_me = gen_ZNE_MitEx(backend=noisy_backend, 
                       noise_scaling_list=[9,7,5,3], 
                       fit_type=Fit.exponential, 
                       show_fit=True)

In [ ]:
from pytket.utils import QubitPauliOperator
from qermit import AnsatzCircuit, SymbolsDict, ObservableExperiment, ObservableTracker
import seaborn as sns 

sns.set_style("whitegrid")

qpo = QubitPauliOperator({qps:1})
ops_exp = ObservableExperiment(AnsatzCircuit(rand_circ, 10000, SymbolsDict()), ObservableTracker(qpo))
zne_me.run([ops_exp])

## ZNE Options and Design

In [ ]:
zne_me.get_task_graph()

## SPAM Error-Mitigation With Qermit

In [ ]:
from qermit.spam import gen_UnCorrelated_SPAM_MitRes

spam_mr = gen_UnCorrelated_SPAM_MitRes(noisy_backend, n_shots)
spam_mr.get_task_graph()

## Combining Error-Mitigation Schemes

In [ ]:
from qermit.zero_noise_extrapolation import gen_ZNE_MitEx, Fit, Folding

zne_spam_me = gen_ZNE_MitEx(backend=noisy_backend, 
                       noise_scaling_list=[9,7,5,3], 
                       fit_type=Fit.exponential, 
                       folding_type=Folding.circuit,
                       show_fit=True,
                       experiment_mitres=spam_mr)
zne_spam_me.get_task_graph()

In [ ]:
zne_spam_me.run([ops_exp])

## Clifford Data Regression

In [ ]:
from qermit.clifford_noise_characterisation import gen_CDR_MitEx

noiseless_backend = AerBackend()

cdr_mitex = gen_CDR_MitEx(device_backend = noisy_backend,
                      simulator_backend = noiseless_backend,
                      n_non_cliffords = 5,
                      n_pairs = 3,
                      total_state_circuits = 50)

cdr_mitex.get_task_graph()

In [ ]:
cdr_mitex.run([ops_exp])